In [2]:
# !pip install tensorflow==2.16.1
# pip install numpy==1.26.4
# 运行2遍
import json
import os
import sys
import asyncio
import argparse
from collections import defaultdict
import time
import subprocess

os.environ["JAX_PLATFORMS"] = "cpu"
from etils import epath
import json
import base64

import numpy as np
import jax.numpy as jnp
import jax
import orbax
import orbax.checkpoint as ocp
from etils import epath
from jax.sharding import PartitionSpec as PS
from flax.traverse_util import flatten_dict, unflatten_dict


METADATA_FILE = '_METADATA'
_CHECKPOINT_FILE = 'checkpoint'


read_dir = 'gs://llm_base_models_europe-west4/v5p_256/7B/PileDCSlimLlama7B32Kx4x256x1v5p_0713/checkpoints/440000/state'
save_dir = 'gs://llm_base_models_europe-west4/v5p_256/7B/xm_E8x7B_OnlyUnshareWithMgate_selfinit_1205/checkpoints/'
# read_dir = 'gs://llm_base_models_europe-west4/v5p_256/7B/xm_M8x7B_WithMgateNoshareCopymlp_1129/checkpoints/1400/state'

read_dir = epath.Path(read_dir) 
# save_dir = epath.Path(save_dir)

metadata_path = read_dir / METADATA_FILE
back_metadata_path = read_dir / f'{METADATA_FILE}.back'
try:
    metadata_path.rename(back_metadata_path)
except:
    pass
metadata_path.unlink(missing_ok=True) # delete
structure_path = read_dir / _CHECKPOINT_FILE
msgpack = ocp.aggregate_handlers.MsgpackHandler(0)
structure = msgpack.deserialize(structure_path)
# backup original checkpoint fil
back_structure_path = read_dir / 'checkpoint_back'
back_structure = structure.copy()
if not back_structure_path.exists():
    asyncio.run(msgpack.serialize(back_structure_path, item=back_structure))
print(f'Old structure file keys: {structure.keys()}')
remove_keys = ['opt_state', 'step'] # select the weight name you don't want to load, all weight name: opt_state, step, params
_ = [structure.pop(key) for key in remove_keys if key in structure]
print(f'New structure file keys: {structure.keys()}')
asyncio.run(msgpack.serialize(structure_path, item=structure))  # rewrite struct file

# load model based struct, note: axes must same as training
mesh_axes = ['data', 'stage', 'fsdp', 'fsdp_transpose', 'sequence', 'tensor', 'autoregressive']
devices = np.asarray(jax.devices()).reshape([1] * len(mesh_axes))
mesh = jax.sharding.Mesh(devices, mesh_axes)
sharding = jax.sharding.NamedSharding(mesh, PS()) # Sharding is None because we use cpu to load weights
weight_dtype = jnp.bfloat16 # set restore weights dtype
restore_args = {}
for k, v in flatten_dict(structure).items():
    restore_args[k] =  ocp.ArrayRestoreArgs(restore_type=jax.Array, dtype=weight_dtype, sharding=sharding)
restore_args = unflatten_dict(restore_args)
ckptr = ocp.Checkpointer(ocp.PyTreeCheckpointHandler())
w = ckptr.restore(read_dir, args=ocp.args.PyTreeRestore(restore_args=restore_args))
structure_path = read_dir / _CHECKPOINT_FILE
# rewrite struct file, otherwise occur error when continue training
asyncio.run(msgpack.serialize(structure_path, item=back_structure))
while 'params' in w:
    w = w['params']
xm3p5_w = {'.'.join(k): np.array(v) for k, v in flatten_dict(w).items()}

try:
    back_metadata_path.rename(metadata_path)
except:
    pass

2024-12-05 11:56:44.849293: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-05 11:56:45.140395: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-05 11:56:45.142475: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-05 11:56:47.307694: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Old structure file keys: dict_keys(['params'])
New structure file keys: dict_keys(['params'])


I0000 00:00:1733399838.006031   20636 gcs_resource.cc:109] Using default AdmissionQueue with limit 32
I0000 00:00:1733399838.019827   21457 google_auth_provider.cc:180] Running on GCE, using service account 887571727717-compute@developer.gserviceaccount.com


In [3]:
## 基于dense模型保存和moe相同名字的参数
def convert_to_jnp(params, remove_keys=[]):
    convert_params = {}
    for k, v in params.items():
        r = 0
        for remove_key in remove_keys:
            if remove_key in k: 
                r = 1
                break
        if r: continue
        k = tuple(k.split('.'))
        convert_params[k] = v
        # convert_params[k] = jnp.array(v).astype(jnp.bfloat16)
    for k, v in convert_params.items():
        print(k, v.shape, v.dtype)
    return convert_params


def save_params(step, save_dir, params):
    item = {
            'state': orbax.checkpoint.AsyncCheckpointer(orbax.checkpoint.PyTreeCheckpointHandler(use_ocdbt=False)),
                    }
    unflatten_params = unflatten_dict(params)
    for k, v in params.items():
        print(k, v.shape, v.dtype)
    mngr = orbax.checkpoint.CheckpointManager(save_dir, item)
    if 'params' not in unflatten_params: unflatten_params = {'params': unflatten_params}
    mngr.save(step, items={'state': {'params': unflatten_params}})

params_save_step = 0
# convert_params = convert_to_jnp(xm3p5_w, remove_keys=['mlp_'])
# save_params(params_save_step, save_dir, convert_params)

In [7]:
def nd_dense_init(scale, mode, distribution):
  """Initializer with in_axis, out_axis set at call time."""

  def init_fn(key, shape, dtype, in_axis, out_axis):
    fn = jax.nn.initializers.variance_scaling(scale, mode, distribution, in_axis, out_axis)
    return fn(key, shape, dtype)

  return init_fn

init_func = nd_dense_init(1.0, "fan_in", "truncated_normal")


## moe部分的参数保存，保存后在bucket后台人工进行转移
start_time = time.time()
unshared_experts = 8

scale = 1
mlp_dim = 5632 // scale
model_dim = 4096 // scale
copy_dim = mlp_dim // unshared_experts

# fp16_dtype = np.dtype('float16')
# 4个子层
moe_params = {}
example_params = {}
for k, v in xm3p5_w.items():
    v = jnp.array(v).astype(jnp.bfloat16)
    ev = jnp.array([100]).astype(jnp.bfloat16)
    if 'decoder.layers.mlp_' in k:
        mlp_inx = k.find('mlp_')
        l = k[mlp_inx+4: mlp_inx+5]
        unshared_mlp = k.replace('decoder.layers.mlp_', 'decoder.layers.unshared_mlp_')
        unshared_mlp = unshared_mlp.replace('.kernel', '')
        if 'mgate' in unshared_mlp:
            init_w = v.transpose(1, 0, 2)[None].repeat(unshared_experts, 0)
        else:
            v = v.transpose(1, 0, 2)
            # unshared: unshared_experts * 12 * model_dim * mlp_dim,  mlp: model_dim * 12 * mlp_dim
            unshared_mlp_w = []
            if '.wo.' in k:
                copy_w = v.reshape(12, unshared_experts, 1, copy_dim, model_dim).repeat(unshared_experts, 2).reshape(
                    12, unshared_experts, -1, model_dim).transpose(1, 0, 2, 3)
            else:
                copy_w = v.reshape(12, model_dim, unshared_experts, 1, copy_dim).repeat(unshared_experts, 3).reshape(
                    12, model_dim, unshared_experts, -1).transpose(2, 0, 1, 3)
            v = v[None] # extend dim
            init_w = (copy_w + v) / 2
            
        moe_params[unshared_mlp] = init_w
        example_params[unshared_mlp] = ev
        if 'mgate' in unshared_mlp:
            router_gate = unshared_mlp.replace('mgate', 'router_gate.kernel')
            # init_v = v[..., :unshared_experts]
            # 为什么用这个初始化的时候，模型被load进cpu。取v的值的时候，被load进tpu，啥原因？
            init_v = init_func(key=jax.random.PRNGKey(9876), shape=v[...,:unshared_experts].shape, dtype=jnp.bfloat16, in_axis=0, out_axis=2)
            print(f'router_gate: {router_gate} init_v: {init_v.shape}')
            moe_params[router_gate] = jnp.array(np.array(init_v))
            example_params[router_gate] = ev
    else:
        example_params[k] = ev
        
            
moe_params = {tuple(k.split('.')): v for k, v in moe_params.items()}
example_params = {tuple(k.split('.')): v for k,v in example_params.items()}

for k, v in moe_params.items():
    print(k, v.shape, v[0].sum(), v[1].sum())

moe_save_step = params_save_step + 1
example_step = params_save_step + 2

save_params(moe_save_step, save_dir, moe_params)
save_params(example_step, save_dir, example_params)

router_gate: decoder.layers.unshared_mlp_0.router_gate.kernel init_v: (4096, 12, 8)
router_gate: decoder.layers.unshared_mlp_1.router_gate.kernel init_v: (4096, 12, 8)
router_gate: decoder.layers.unshared_mlp_2.router_gate.kernel init_v: (4096, 12, 8)
router_gate: decoder.layers.unshared_mlp_3.router_gate.kernel init_v: (4096, 12, 8)
('decoder', 'layers', 'unshared_mlp_0', 'mgate') (8, 12, 4096, 44) -2.78125 -2.78125
('decoder', 'layers', 'unshared_mlp_0', 'router_gate', 'kernel') (4096, 12, 8) 0.0144043 -0.034668
('decoder', 'layers', 'unshared_mlp_0', 'wi_0') (8, 12, 4096, 5632) 5312 4192
('decoder', 'layers', 'unshared_mlp_0', 'wi_1') (8, 12, 4096, 5632) 968 1088
('decoder', 'layers', 'unshared_mlp_0', 'wo') (8, 12, 5632, 4096) -184 147
('decoder', 'layers', 'unshared_mlp_1', 'mgate') (8, 12, 4096, 44) 50 50
('decoder', 'layers', 'unshared_mlp_1', 'router_gate', 'kernel') (4096, 12, 8) 0.0144043 -0.034668
('decoder', 'layers', 'unshared_mlp_1', 'wi_0') (8, 12, 4096, 5632) 5984 6272


In [9]:
import subprocess

source_dir = str(save_dir).rstrip('/')
target_dir = str(save_dir).rstrip('/')

command = f'gsutil cp {source_dir}/{example_step}/state/_METADATA {target_dir}/{params_save_step}/state/ '
r = subprocess.run(command, stdout=subprocess.PIPE, shell=True)

command = f'gsutil cp {source_dir}/{example_step}/state/checkpoint {target_dir}/{params_save_step}/state/ '
r = subprocess.run(command, stdout=subprocess.PIPE, shell=True)

command = f'gsutil -m cp -r {source_dir}/{moe_save_step}/state/params.params* {target_dir}/{params_save_step}/state/ '
r = subprocess.run(command, stdout=subprocess.PIPE, shell=True)

Copying gs://llm_base_models_europe-west4/v5p_256/7B/xm_E8x7B_OnlyUnshareWithMgate_selfinit_1205/checkpoints/2/state/_METADATA [Content-Type=application/octet-stream]...
/ [1 files][ 27.8 KiB/ 27.8 KiB]                                                
Operation completed over 1 objects/27.8 KiB.                                     
Copying gs://llm_base_models_europe-west4/v5p_256/7B/xm_E8x7B_OnlyUnshareWithMgate_selfinit_1205/checkpoints/2/state/checkpoint...
/ [1 files][  6.2 KiB/  6.2 KiB]                                                
Operation completed over 1 objects/6.2 KiB.                                      
Copying gs://llm_base_models_europe-west4/v5p_256/7B/xm_E8x7B_OnlyUnshareWithMgate_selfinit_1205/checkpoints/1/state/params.params.decoder.layers.unshared_mlp_0.mgate/.zarray [Content-Type=application/octet-stream]...
Copying gs://llm_base_models_europe-west4/v5p_256/7B/xm_E8x7B_OnlyUnshareWithMgate_selfinit_1205/checkpoints/1/state/params.params.decoder.layers.unshared_

In [10]:
# 基于tpu type 构建_sharding文件
import base64

def decode_base64(encoded_str):
    decoded_bytes = base64.b64decode(encoded_str)
    decoded_str = decoded_bytes.decode('utf-8')
    return decoded_str

def encode_base64(decoded_str):
    # decoded_str = "opt_state.mu.params.token_embedder.embedding"
    encoded_string = base64.b64encode(decoded_str.encode('utf-8')).decode('utf-8')
    return encoded_string

'''
_sharding文件格式如下：
{
  b3B0X3N0YXRlLm11LnBhcmFtcy50b2tlbl9lbWJlZGRlci5lbWJlZGRpbmc=': {'sharding_type': 'NamedSharding',
  'shape': [1, 1, 4, 1, 1, 1, 1],
  'axis_names': ['data', 'stage', 'fsdp', 'fsdp_transpose', 'sequence', 'tensor','autoregressive'],
  'partition_spec': [['tensor', 'autoregressive'], ['fsdp', 'fsdp_transpose', 'sequence']],
   2: 4},
   ...
   }
   '''
# moe sharding
_sharding_path = 'gs://llm_base_models_us-east5/v5p_256/7B/xm_45x7B_moe_1017/xm3p5_moe_params_no_opt_v5p_64_sharding.copy'
_sharding_path = epath.Path(_sharding_path)
# 读取已有的_sharding文件
with _sharding_path.open('r') as f:
    _sharding = json.load(f)

tpu_type = 'v5p-32'
core_nums = int(tpu_type.split('-')[-1])
if 'v3' not in tpu_type:
    core_nums = core_nums // 2
print(f'core_nums: {core_nums}')
updated_sharding = {}
for k, v in _sharding.items():
    if isinstance(v, str):
        v = json.loads(v)
    v['shape'][2] = core_nums
    if len(v['shape']) == 8:
        v['shape'] = v['shape'][:-1]
    base_k = decode_base64(k)
    if 'opt_state' in base_k or 'layers.mlp' in base_k: 
        print(f'remove key: {base_k}')
        continue
    updated_sharding[k] = json.dumps(v)
    if 'unshared_mlp_0' in base_k: # 因为已有的sharding文件是隔层moe，因此需要进行扩展
        unshared_mlp_1 = base_k.replace('unshared_mlp_0', 'unshared_mlp_1')
        unshared_mlp_3 = base_k.replace('unshared_mlp_0', 'unshared_mlp_3')
        encode_unshared_mlp_1 = encode_base64(unshared_mlp_1)
        encode_unshared_mlp_3 = encode_base64(unshared_mlp_3)
        print(f'encode_unshared_mlp_1: {unshared_mlp_1}')
        print(f'encode_unshared_mlp_3: {unshared_mlp_3}')
        
        updated_sharding[encode_unshared_mlp_1] = json.dumps(v)
        updated_sharding[encode_unshared_mlp_3] = json.dumps(v)
    
updated_sharding_path = f'{save_dir}/{params_save_step}/state/_sharding'

updated_sharding_path = epath.Path(updated_sharding_path)
with updated_sharding_path.open('w') as f:
    json.dump(updated_sharding, f)

core_nums: 16
remove key: opt_state.mu.params.token_embedder.embedding
remove key: opt_state.mu.params.decoder.decoder_norm.scale
remove key: opt_state.mu.params.decoder.layers.unshared_mlp_0.wi_0
remove key: opt_state.mu.params.decoder.layers.unshared_mlp_0.wi_1
remove key: opt_state.mu.params.decoder.layers.unshared_mlp_0.wo
remove key: opt_state.mu.params.decoder.layers.unshared_mlp_0.mgate
remove key: opt_state.mu.params.decoder.layers.unshared_mlp_0.router_gate.kernel
remove key: opt_state.mu.params.decoder.layers.unshared_mlp_2.wi_0
remove key: opt_state.mu.params.decoder.layers.unshared_mlp_2.wi_1
remove key: opt_state.mu.params.decoder.layers.unshared_mlp_2.wo
remove key: opt_state.mu.params.decoder.layers.unshared_mlp_2.mgate
remove key: opt_state.mu.params.decoder.layers.unshared_mlp_2.router_gate.kernel
remove key: opt_state.mu.params.decoder.layers.mlp_0.wi_0.kernel
remove key: opt_state.mu.params.decoder.layers.mlp_0.wi_1.kernel
remove key: opt_state.mu.params.decoder.laye